In [1]:
import numpy as np
import pandas as pd
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import h5py
from PIL import Image, ImageDraw
import nrrd

from matplotlib import pyplot as plt
%matplotlib notebook

# Load reference space

In [2]:
mcc = MouseConnectivityCache(resolution=10)
rsp = mcc.get_reference_space()
rsp.remove_unassigned();

2020-04-29 14:04:33,829 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/annotation_10.nrrd


In [3]:
name_map = rsp.structure_tree.get_name_map()

In [4]:
avg,meta = mcc.get_template_volume()

2020-04-29 14:07:09,268 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/average_template/average_template_10.nrrd


In [5]:
avg.shape

(1320, 800, 1140)

# Load data

In [6]:
data = pd.read_csv('./new_data_121719.csv')

FileNotFoundError: [Errno 2] File b'./new_data_121719.csv' does not exist: b'./new_data_121719.csv'

In [7]:
IDs = data['image id'].values

# Get experiment injection centroids

In [8]:
exp = mcc.get_experiments(dataframe=True)

In [13]:
subset = exp.loc[IDs,['injection_structures','injection_x','injection_y','injection_z']]

# Get flat map lookup tables

In [24]:
with h5py.File('//allen/aibs/ccf/2017_integration/handoff/streamlines/surface_paths_10.h5') as f:
    path_lookup = f['volume lookup'][()]
    paths = f['paths'][()]

In [25]:
with h5py.File('//allen/aibs/ccf/2017_integration/handoff/view_lookup/top.h5') as f:
    top_lookup = f['view lookup'][()]
    top_size = f.attrs['view size']

In [31]:
# This gives us the path indeces that point to the side
path_indeces = path_lookup.flat[top_lookup[:,1]]

# Now we can subsample the paths
top_paths = paths[path_indeces,:]

In [26]:
# Flat map lookup table
flatmap_lookup = h5py.File(r'//allen/aibs/ccf/Maitham/butterfly_projections/flatmap_lookup.h5')
flat_paths = flatmap_lookup['flat_paths'][()]
flat_map = flatmap_lookup['flat_map'][()]
fsize = flatmap_lookup.attrs['size']
flatmap_lookup.close()

In [61]:
path_indeces = path_lookup.flat[flat_paths]
flat_paths = paths[path_indeces,:]

In [5]:
def get_top_centroid_coord(x,y,z,volume_shape,flat_shape):
    point_blank = np.zeros(volume_shape,dtype=np.uint8)
    flat_blank = np.zeros(flat_shape,dtype=np.uint8)
    
    point_blank[x-1:x+2,y-1:y+2,z-1:z+2] = 1
    point_blank = np.flip(point_blank,axis=2)
    
    for streamline in np.arange(top_paths.shape[0]):
        flat_blank.flat[top_lookup[streamline,0]] = \
                max(point_blank.flat[top_paths[streamline,:]])
    
    return np.round(np.mean(np.where(flat_blank),axis=1)).astype(int)

In [6]:
def get_flat_centroid_coord(x,y,z,volume_shape,flat_shape):
    point_blank = np.zeros(volume_shape,dtype=np.uint8)
    flat_blank = np.zeros(flat_shape,dtype=np.uint8)
    
    point_blank[x-1:x+2,y-1:y+2,z-1:z+2] = 1
    point_blank = np.flip(point_blank,axis=2)
    
    for streamline in np.arange(flat_paths.shape[0]):
        flat_blank.flat[flat_map[streamline]] = max(point_blank.flat[flat_paths[streamline,:]])
    
    return np.round(np.mean(np.where(flat_blank),axis=1)).astype(int)

In [7]:
(x,y,z) = (subset.iloc[0,1:]/10).astype(int)

NameError: name 'subset' is not defined

In [208]:
x,y,z

(559, 139, 746)

In [108]:
top_centroid = get_top_centroid_coord(x,y,z,avg.shape,top_size)

In [63]:
flat_centroid = get_flat_centroid_coord(x,y,z,avg.shape,fsize)

In [205]:
top_centroid

array([569, 393])

In [206]:
flat_centroid

array([1047,  505])

In [65]:
centroids = []
for i,row in subset.iterrows():
    x,y,z = (row[['injection_x','injection_y','injection_z']]/10).astype(int)
    top_centroid = get_top_centroid_coord(x,y,z,avg.shape,top_size)
    flat_centroid = get_flat_centroid_coord(x,y,z,avg.shape,fsize)
    centroids.append({'ID':i,'top_x':top_centroid[0],'top_y':top_centroid[1],'flat_x':flat_centroid[0],'flat_y':flat_centroid[1]})   
cent_df = pd.DataFrame(centroids)

C:\Users\maithamn\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\maithamn\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


# Fix missing values

In [ ]:
temp = cent_df.set_index('ID')

In [150]:
for i,row in subset.iterrows():
    if i in (112952510,141603190):
        x,y,z = (row[['injection_x','injection_y','injection_z']]/10).astype(int)
        # I first modified this function to mark a larger region for the injection centroid
        top_centroid = get_top_centroid_coord(x,y,z,avg.shape,top_size)
        
        temp.loc[i,['top_x','top_y']] = top_centroid
        

In [152]:
cent_df = temp.copy()

In [156]:
img = Image.new('RGBA',(fsize[1],fsize[0]),(255,255,255,0))
draw = ImageDraw.Draw(img)
for i,row in cent_df.iterrows():
    centroid = row[['flat_x','flat_y']]


    draw.ellipse((centroid[0]-3,centroid[1]-3,centroid[0]+3,centroid[1]+3),fill='black')
#     img.save('./new_results_for_julie/{}f.png'.format(i))

In [158]:
img.save('./flat_map_results.png')

In [168]:
img = Image.new('RGBA',(top_size[0],top_size[1]),(255,255,255,0))
draw = ImageDraw.Draw(img)
for i,row in cent_df.iterrows():
    centroid = row[['top_x','top_y']]


    draw.ellipse((centroid[0]-3,centroid[1]-3,centroid[0]+3,centroid[1]+3),fill='black')
#     img.save('./new_results_for_julie/{}.png'.format(i))

In [170]:
img.save('./top_view_results.png')

In [165]:
data.iloc[:17,0].values

array([100141273, 100141563, 100141780, 126909424, 127084296, 180720175,
       272697944, 584903636, 100141454, 112952510, 141602484, 141603190,
       157710335, 180709942, 180719293, 180916954, 585025284], dtype=int64)

In [171]:
cent_df.to_json('./new_results.json')

# Add labels

In [177]:
from PIL import ImageFont

In [196]:
font = ImageFont.truetype("arial",size=15)

In [201]:
img = Image.new('RGBA',(fsize[1],fsize[0]),(255,255,255,0))
draw = ImageDraw.Draw(img)
for i,row in cent_df.iterrows():
    centroid = row[['flat_x','flat_y']]


    draw.text((centroid[0]-3,centroid[1]-3,centroid[0]+3,centroid[1]+3),"{}".format(i),fill='blue',font=font)
#     img.save('./new_results_for_julie/{}f.png'.format(i))

In [190]:
img.show()

In [202]:
img.save('./flat_ids.png')

In [199]:
img = Image.new('RGBA',(top_size[0],top_size[1]),(255,255,255,0))
draw = ImageDraw.Draw(img)
for i,row in cent_df.iterrows():
    centroid = row[['top_x','top_y']]


    draw.text((centroid[0]-3,centroid[1]-3,centroid[0]+3,centroid[1]+3),"{}".format(i),fill='blue',font=font)
#     img.save('./new_results_for_julie/{}.png'.format(i))

In [200]:
img.save('./top_ids.png')